In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
from seq.model import Event, EventGraph

In [16]:
data = np.random.randint(1, 5, (10_000, 32))
data.shape

(10000, 32)

In [17]:
G = EventGraph()
for position in range(data.shape[1]):
  u = np.unique(data[:, position])

  for value in u:
    G.add_event(
      Event(
        value,
        position,
        np.argwhere(data[:, position] == value).flatten().tolist(),
      )
    )

In [18]:
max_weight_edge = min(G.edges(data=True), key=lambda x: x[2]["weight"])

In [19]:
max_weight_edge

(Token(Position=2, Value=1, #Indices=1087),
 Token(Position=15, Value=2, #Indices=1040),
 {'weight': 84})